<a href="https://colab.research.google.com/github/afvallejo/SIG/blob/master/Copy_of_Differential_expression_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfering annotattion to filtered data

In [0]:
adata1=sc.read(filteredsample)

In [0]:
adata1.obs["annotated"]=adata.obs["predicted1"]

In [0]:
filteredannotated=samplename+"_filtered_annotated.h5ad"
adata1.write(filteredannotated)

# Install packages 

In [0]:
%%time
!pip install MulticoreTSNE scanpy[louvain] MulticoreTSNE anndata2ri scvelo leidenalg bbknn rpy2==3.3.2

In [0]:
!pip install diffxpy

In [0]:
%load_ext rpy2.ipython

install.packages("BiocManager")

BiocManager::install()

a

q()

n



In [0]:
#@title
!R

In [0]:
%%time
%%R
options(Ncpus = 4)
options(repos=structure(c(CRAN="http://cloud.r-project.org/")))
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
check.packages <- function(pkg){
    new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
    if (length(new.pkg)) 
        BiocManager::install(new.pkg, dependencies = TRUE)
    sapply(pkg, require, character.only = TRUE)
}

# Usage
packages<-c("MAST", "scater")
check.packages(packages)

# set up WD

 # load Libraries

In [0]:
import scanpy as sc

# numpy et al.
import numpy as np
import scipy as sp
import pandas as pd
import gc
import bbknn

# R integration
from rpy2.robjects.packages import importr

from rpy2.robjects.vectors import StrVector, FloatVector, ListVector
import rpy2.robjects as ro
import anndata2ri

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import seaborn as sb
import re
import scipy as sp
import datetime, time
import diffxpy.api as de
sc.settings.verbosity = 3               # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.logging.print_version_and_date()


import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri, numpy2ri, r



# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython


# Download the data

In [0]:
!gdown --id 13Wd7kxAn7vHcNsbEZE5H--3Du9dPkVza

In [0]:
adata=sc.read('PBMC_IFN_singler.h5ad')
adata

In [0]:
sb.set_context('paper')
pl.rcParams['figure.figsize']=(10,10)

sc.pl.umap(adata,color=['cluster', 'Donor','stim' ],legend_loc='on data')

In [0]:
marker_genes = ['CD79A', 'MS4A1', 'CD8A', 'CD8B', 'GNLY', 'NKG7','IL7R','LYZ', 'CD14','FCGR3A', 'MS4A7', 'FCER1A', 'CST3',
                'PPBP','LGALS3', 'S100A8', 'KLRB1', 'HLA-DRA']

In [0]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(4,4) #rescale figures
sc.pl.umap(adata=adata, color=marker_genes,color_map='Reds', use_raw=True)

# Differential expression analysis

## Pseudotime on Monocytes

In [0]:
subset=adata[adata.obs['leiden'].isin(['0','9'])]
adata_mono=subset[subset.obs['stim'].isin(['CTRL'])]
adata_mono

In [0]:

sc.pl.umap(adata_mono,color='leiden')

In [0]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)

sc.tl.pca(adata_mono, svd_solver='arpack')
sc.tl.diffmap(adata_mono)
#sc.pp.neighbors(adata_mono, n_neighbors=10, random_state=42,use_rep='X_diffmap')
sc.pp.neighbors(adata_mono)

In [0]:
sc.tl.leiden(adata,random_state=42, resolution = 0.5)
sc.pl.umap(adata_mono,color='leiden')

In [0]:

umap_0 = [term[0] for term in adata_mono.obsm['X_umap']]
umap_1 = [term[1] for term in adata_mono.obsm['X_umap']]

In [0]:
adata_mono.uns['iroot'] = np.flatnonzero(umap_1== min(umap_1))[0]
sc.tl.dpt(adata = adata_mono)

In [0]:
pl.rcParams['figure.figsize']=(7,7)
sc.pl.umap(adata_mono, color=['dpt_pseudotime', 'leiden'])

In [0]:
pl.rcParams['figure.figsize']=(7,7)
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'leiden'], components=['1,2'])
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'leiden'], components=['1,3'])

### Run a differential test on the two groups of monocytes in order to determine characteristic genes.

In [0]:
sc.tl.rank_genes_groups(adata_mono, groupby='leiden', 
                        groups= ['9'], reference='0', rankby_abs=True)

In [0]:
pl.rcParams['figure.figsize']=(10,5)
sc.pl.rank_genes_groups(adata_mono, size=10, n_genes=30)

In [0]:
pl.rcParams['figure.figsize']= (15,5)
sc.pl.rank_genes_groups_violin(adata_mono)

In [0]:
mono_genes = [idx[1][0] for idx in enumerate(adata_mono.uns['rank_genes_groups']['names'])]

## Partition-based graph abstraction (PAGA)

In [0]:
pl.rcParams['figure.figsize']=(5,5)
sc.tl.paga(adata_mono, groups='leiden')
sc.pl.paga_compare(adata_mono)

In [0]:
pl.rcParams['figure.figsize']=(20,10)
sc.pl.paga_path(adata_mono, nodes=['9', '0'], 
                keys=mono_genes[:25],n_avg=10, use_raw=False)

## t-test

In [0]:
subset=adata[adata.obs['cluster'].isin(['CD14 Mono'])]
subset

In [0]:
sc.pp.neighbors(subset, n_pcs=30,n_neighbors=30,random_state=42)

sc.tl.leiden(subset, resolution = 0.5)
sc.tl.umap(subset)

sc.pl.umap(subset, color = ['cluster', 'Donor','stim'])

In [0]:
sc.tl.rank_genes_groups(subset, groupby='stim', 
                        groups= ['STIM'], reference='CTRL', rankby_abs=True)

In [0]:
pl.rcParams['figure.figsize']=(10,5)
sc.pl.rank_genes_groups(subset, size=10, n_genes=50)

In [0]:
markers=pd.DataFrame(subset.uns['rank_genes_groups']['names']).head(30)
markers

In [0]:
pl.rcParams['lines.linewidth'] = 0.1
sc.pl.rank_genes_groups_violin(subset,  n_genes=20, jitter=False)

# Difxpy

## T-test

In [0]:
test_tt = de.test.t_test(data=subset,grouping="stim")

In [0]:
test_tt.summary().iloc[:10,:]

In [0]:
DEG=test_tt.gene_ids[1:10]
DEG

In [0]:
pl.rcParams['lines.linewidth'] = 0.1
sc.pl.rank_genes_groups_violin(subset, gene_names=DEG,  jitter=False)

# MAST

In [0]:
#Create new Anndata object for use in MAST with non-batch corrected data as before
adata_test = subset.copy()
adata_test.X = subset.raw.X
adata_test.obs['n_genes'] = (adata_test.X > 0).sum(1) # recompute number of genes expressed per cell

In [0]:
adata_test.shape

In [0]:
%%R -i adata_test 
suppressPackageStartupMessages(library(scater))
suppressPackageStartupMessages(library(MAST))

adata_test<-logNormCounts(adata_test)
sca = SceToSingleCellAssay(adata_test)




In [0]:
%%time
%%R
colData(sca)$n_genes = scale(colData(sca)$n_genes)

#Filter out non-expressed genes in the subsets
print("Dimensions before filtering:")
print(dim(sca))
print("")
sca_filt = sca[rowSums(assay(sca)) != 0, ]
print("Dimensions after filtering:")
print(dim(sca_filt))

#Define & run hurdle model 
zlmCond_CTR <- zlm(formula = ~ stim + n_genes, sca=sca_filt)

In [0]:
%%time
%%R -o IFN_de
options(Ncpus = 4)
summaryCond_CTR <- summary(zlmCond_CTR, doLRT='stimSTIM')
summaryDt_CTR <- summaryCond_CTR$datatable

result_CTR <- merge(summaryDt_CTR[contrast=='stimSTIM' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                 summaryDt_CTR[contrast=='stimSTIM' & component=='logFC', .(primerid, coef)],
                 by='primerid') #logFC coefficients

#Correct for multiple testing (FDR correction) and filtering
result_CTR[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')]
IFN_de = result_CTR[result_CTR$FDR<0.01,, drop=F]
IFN_de = IFN_de[order(IFN_de$FDR),]

In [0]:
print(IFN_de.shape)
IFN_de[:20]

In [0]:
#Volcano plot of results
IFN_de['-logQ'] = -np.log(IFN_de['FDR'])
lowqval_de = IFN_de.loc[IFN_de['-logQ'] > 200]
other_de = IFN_de.loc[IFN_de['-logQ'] < 200]

fig, ax = pl.subplots()
sb.regplot(other_de['coef'], other_de['-logQ'], fit_reg=False, scatter_kws={'s':6})
sb.regplot(lowqval_de['coef'], lowqval_de['-logQ'], fit_reg=False, scatter_kws={'s':6})
ax.set_xlabel("log2 FC", fontsize=20)
ax.set_ylabel("-log Q-value", fontsize=20)
ax.tick_params(labelsize=15)

# Label names and positions
x = [i-0.2 for i in lowqval_de['coef']]
y = [i+10 for i in lowqval_de['-logQ']]
labels = lowqval_de['primerid']

# Show only some labels to avoid overcrowding the figure
to_remove = np.where([i < 230 for i in y])[0]
labels = ["" if i in to_remove else lab for i,lab in enumerate(labels) ]

#Move up two labels
y = [y[i]+10 if txt == 'Krt8' else y[i] for i,txt in enumerate(labels)]
y = [y[i]+20 if txt == 'Cd9' else y[i] for i,txt in enumerate(labels)]

#Move down one label
y = [y[i]-20 if txt == 'Phgr1' else y[i] for i,txt in enumerate(labels)]

for i,txt in enumerate(labels):
    ax.annotate(txt, (x[i], y[i]))
pl.show()

In [0]:
IFN_de.to_csv("DEG_allcells_CTR_vs_IFN.csv")